In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/bert-en-uncased-l12-h768-a122/saved_model.pb
/kaggle/input/bert-en-uncased-l12-h768-a122/assets/vocab.txt
/kaggle/input/bert-en-uncased-l12-h768-a122/variables/variables.index
/kaggle/input/bert-en-uncased-l12-h768-a122/variables/variables.data-00000-of-00001
/kaggle/input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl
/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl
/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12/saved_model.pb
/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12/assets/vocab.txt
/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12/variables/variables.index
/kaggle/input/bert-base-from-tfhub/bert_en_uncased_L-12_H-768_A-12/variables/variables.data-00000-of-00001


In [2]:
from keras.models import *
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler
import time
import pickle
import re
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [4]:
CRAWL_EMBEDDING_PATH = '../input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl'
GLOVE_EMBEDDING_PATH = '../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl'

In [5]:
sample_submission_df = pd.read_csv("../input/google-quest-challenge/sample_submission.csv")
test_df = pd.read_csv("../input/google-quest-challenge/test.csv")
train_df = pd.read_csv("../input/google-quest-challenge/train.csv")

In [6]:
sample_submission_df.shape

(476, 31)

In [7]:
train_df.shape

(6079, 41)

In [8]:
# Exploring Data
samp_id = 9
print('Question Title: %s \n' % train_df['question_title'].values[samp_id])
print('Question Body: %s \n' % train_df['question_body'].values[samp_id])
print('Answer: %s' % train_df['answer'].values[samp_id])

Question Title: How do you grapple in Dead Rising 3? 

Question Body: I just started playing Dead Rising 3 on the Xbox One. I got to the first grapple, and I can't figure out what I'm supposed to do. The top of the screen says "Perform a gesture to escape a grapple" - what does "gesture" mean in this context? Is it something to do with Kinect?
 

Answer: You can also switch gesture based grapple escapes off in the Kinect settings of Dead Rising 3 (along with all the rest of the Kinect features) so that you have to do QTE type button presses instead.



In [9]:
train_users = set(train_df['question_user_page'].unique())
test_users = set(test_df['question_user_page'].unique())

print('Unique users in train set: %s' % len(train_users))
print('Unique users in test set: %s' % len(test_users))
print('Users in both sets: %s' % len(train_users & test_users))
print('What users are in both sets? %s' % list(train_users & test_users))

Unique users in train set: 3422
Unique users in test set: 474
Users in both sets: 1
What users are in both sets? ['https://salesforce.stackexchange.com/users/686']


In [10]:
train_df.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [11]:
(train_df.head()).T

,0,1,2,3,4
qa_id,0,1,2,3,5
question_title,What am I losing when using extension tubes in...,What is the distinction between a city and a s...,Maximum protusion length for through-hole comp...,Can an affidavit be used in Beit Din?,How do you make a binary image in Photoshop?
question_body,After playing around with macro photography on...,I am trying to understand what kinds of places...,I'm working on a PCB that has through-hole com...,"An affidavit, from what i understand, is basic...",I am trying to make a binary image. I want mor...
question_user_name,ysap,russellpierce,Joe Baker,Scimonster,leigero
question_user_page,https://photo.stackexchange.com/users/1024,https://rpg.stackexchange.com/users/8774,https://electronics.stackexchange.com/users/10157,https://judaism.stackexchange.com/users/5151,https://graphicdesign.stackexchange.com/users/...
answer,"I just got extension tubes, so here's the skin...",It might be helpful to look into the definitio...,Do you even need grooves? We make several pro...,"Sending an ""affidavit"" it is a dispute between...",Check out Image Trace in Adobe Illustrator. \n...
answer_user_name,rfusca,Erik Schmidt,Dwayne Reid,Y e z,q2ra
answer_user_page,https://photo.stackexchange.com/users/1917,https://rpg.stackexchange.com/users/1871,https://electronics.stackexchange.com/users/64754,https://judaism.stackexchange.com/users/4794,https://graphicdesign.stackexchange.com/users/...
url,http://photo.stackexchange.com/questions/9169/...,http://rpg.stackexchange.com/questions/47820/w...,http://electronics.stackexchange.com/questions...,http://judaism.stackexchange.com/questions/551...,http://graphicdesign.stackexchange.com/questio...
category,LIFE_ARTS,CULTURE,SCIENCE,CULTURE,LIFE_ARTS


In [12]:
#!pip install transformers

In [13]:
train_df['all_text'] = train_df['question_title'] + " [SEP] " + train_df['question_body'] + " [SEP] " + train_df['question_user_name'] + " [SEP] " + train_df['question_user_page'] + " [SEP] " + train_df['question_user_page'] + " [SEP] " + train_df['answer'] + " [SEP] " + train_df['answer_user_name'] + " [SEP] " + train_df['answer_user_page'] + " [SEP] " + train_df['category'] + " [SEP] " + train_df['host'] 

test_df['all_text'] = test_df['question_title'] + " [SEP] " + test_df['question_body'] + " [SEP] " + test_df['question_user_name'] + " [SEP] " + test_df['question_user_page'] + " [SEP] " + test_df['question_user_page'] + " [SEP] " + test_df['answer'] + " [SEP] " + test_df['answer_user_name'] + " [SEP] " + test_df['answer_user_page'] + " [SEP] " + test_df['category'] + " [SEP] " + test_df['host'] 

In [14]:
# You can use this pre-processing steps at later stage

import re
def pre_process(text):
    new_text =re.sub('[0-9]', '', text)
    new_text = re.sub(r"\u200b","",new_text)
    new_text = re.sub(r"\.+",".",new_text)
    new_text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '',new_text, flags=re.MULTILINE)
    new_text = re.sub("'", "", new_text)
    new_text = re.sub(r'↑', '', new_text)
    new_text = re.sub("\t", "", new_text)
    new_text = re.sub("\xa0", "", new_text)
    new_text = re.sub("\(|\)|\[|\]", "", new_text)
    new_text = re.sub("\n", "", new_text)
    new_text = re.sub("\.", "", new_text)
    new_text = re.sub("\,", " ", new_text)
    new_text = re.sub("[/%]", " ", new_text)
    new_text = re.sub('[/%:;]', '', new_text)
    new_text = re.sub(' +', ' ', new_text)
    return new_text

In [15]:
# remove URL's from train and test
train_df['all_text'] = train_df['all_text'].apply(lambda x: re.sub(r'http\S+', '', x))
test_df['all_text'] = test_df['all_text'].apply(lambda x: re.sub(r'http\S+', '', x))

In [16]:
# remove numbers
train_df['all_text'] = train_df['all_text'].str.replace("[0-9]", " ")
test_df['all_text'] = test_df['all_text'].str.replace("[0-9]", " ")

In [17]:
# Adjusting the load_embeddings function, to now handle the pickled dict.

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [18]:
#Below Functions are to check the embedding’s coverage and building vocabulary 
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab


In [19]:
# Lets load the embeddings 
tic = time.time()
glove_embeddings = load_embeddings(GLOVE_EMBEDDING_PATH)
print(f'loaded {len(glove_embeddings)} word vectors in {time.time()-tic}s')

loaded 2196008 word vectors in 13.122509002685547s


In [20]:
# Lets check how many words we got covered 
vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
oov[:20]


Found embeddings for 31.31% of vocab
Found embeddings for  80.93% of all text


[('[SEP]', 54711),
 ('),', 2306),
 ('STACKOVERFLOW', 1253),
 (');', 1009),
 ('=&gt;', 826),
 ('However,', 728),
 ('LIFE_ARTS', 709),
 ('&gt;', 676),
 ('example,', 573),
 ('this:', 537),
 ('(or', 505),
 ("isn't", 498),
 ('-&gt;', 466),
 ('So,', 433),
 ('$$', 426),
 ('},', 420),
 ('&lt;', 415),
 ("won't", 407),
 ('(and', 390),
 ('this?', 383)]

In [21]:
# Lets replace few words which is not covered in our embedding’s.

replaceWords1 = { "won't":"will not","$&@*#":"in most profane vulgar shitty terms","#$&@*#":"shitty",
 "can't":"cannot","aren't": 'are not',
 "Aren't": 'Are not',
 "AREN'T": 'ARE NOT',
 "C'est": "C'est",
 "C'mon": "C'mon",
 "c'mon": "c'mon",
 "can't": 'cannot',
 "Can't": 'Cannot',
 "CAN'T": 'CANNOT',
 "con't": 'continued',
 "cont'd": 'continued',
 "could've": 'could have',
 "couldn't": 'could not',
 "Couldn't": 'Could not',
 "didn't": 'did not',
 "Didn't": 'Did not',
 "DIDN'T": 'DID NOT',
 "don't": 'do not',
 "Don't": 'Do not',
 "DON'T": 'DO NOT',
 "doesn't": 'does not',
 "Doesn't": 'Does not',
 "else's": 'else',
 "gov's": 'government',
 "Gov's": 'government',
 "gov't": 'government',
 "Gov't": 'government',
 "govt's": 'government',
 "gov'ts": 'governments',
 "hadn't": 'had not',
 "hasn't": 'has not',
 "Hasn't": 'Has not',
 "haven't": 'have not',
 "Haven't": 'Have not',
 "he's": 'he is',
 "He's": 'He is',
 "he'll": 'he will',
 "He'll": 'He will',
 "he'd": 'he would',
 "He'd": 'He would',
 "Here's": 'Here is',
 "here's": 'here is',
 "I'm": 'I am',
 "i'm": 'i am',
 "I'M": 'I am',
 "I've": 'I have',
 "i've": 'i have',
 "I'll": 'I will',
 "i'll": 'i will',
 "I'd": 'I would',
 "i'd": 'i would',
 "ain't": 'is not',
 "isn't": 'is not',
 "Isn't": 'Is not',
 "ISN'T": 'IS NOT',
 "it's": 'it is',
 "It's": 'It is',
 "IT'S": 'IT IS',
 "I's": 'It is',
 "i's": 'it is',
 "it'll": 'it will',
 "It'll": 'It will',
 "it'd": 'it would',
 "It'd": 'It would',
 "Let's": "Let's",
 "let's": 'let us',
 "ma'am": 'madam',
 "Ma'am": "Madam",
 "she's": 'she is',
 "She's": 'She is',
 "she'll": 'she will',
 "She'll": 'She will',
 "she'd": 'she would',
 "She'd": 'She would',
 "shouldn't": 'should not',
 "that's": 'that is',
 "That's": 'That is',
 "THAT'S": 'THAT IS',
 "THAT's": 'THAT IS',
 "that'll": 'that will',
 "That'll": 'That will',
 "there's": 'there is',
 "There's": 'There is',
 "there'll": 'there will',
 "There'll": 'There will',
 "there'd": 'there would',
 "they're": 'they are',
 "They're": 'They are',
 "they've": 'they have',
 "They've": 'They Have',
 "they'll": 'they will',
 "They'll": 'They will',
 "they'd": 'they would',
 "They'd": 'They would',
 "wasn't": 'was not',
 "we're": 'we are',
 "We're": 'We are',
 "we've": 'we have',
 "We've": 'We have',
 "we'll": 'we will',
 "We'll": 'We will',
 "we'd": 'we would',
 "We'd": 'We would',
 "What'll": 'What will',
 "weren't": 'were not',
 "Weren't": 'Were not',
 "what's": 'what is',
 "What's": 'What is',
 "When's": 'When is',
 "Where's": 'Where is',
 "where's": 'where is',
 "Where'd": 'Where would',
 "who're": 'who are',
 "who've": 'who have',
 "who's": 'who is',
 "Who's": 'Who is',
 "who'll": 'who will',
 "who'd": 'Who would',
 "Who'd": 'Who would',
 "won't": 'will not',
 "Won't": 'will not',
 "WON'T": 'WILL NOT',
 "would've": 'would have',
 "wouldn't": 'would not',
 "Wouldn't": 'Would not',
 "would't": 'would not',
 "Would't": 'Would not',
 "y'all": 'you all',
 "Y'all": 'You all',
 "you're": 'you are',
 "You're": 'You are',
 "YOU'RE": 'YOU ARE',
 "you've": 'you have',
 "You've": 'You have',
 "y'know": 'you know',
 "Y'know": 'You know',
 "ya'll": 'you will',
 "you'll": 'you will',
 "You'll": 'You will',
 "you'd": 'you would',
 "You'd": 'You would',
 "Y'got": 'You got',
 'cause': 'because',
 "had'nt": 'had not',
 "Had'nt": 'Had not',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how is',
 "I'd've": 'I would have',
 "I'll've": 'I will have',
 "i'd've": 'i would have',
 "i'll've": 'i will have',
 "it'd've": 'it would have',
 "it'll've": 'it will have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "o'clock": 'of the clock',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shan't've": 'shall not have',
 "she'd've": 'she would have',
 "she'll've": 'she will have',
 "should've": 'should have',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "so's": 'so as',
 "this's": 'this is',
 "that'd": 'that would',
 "that'd've": 'that would have',
 "there'd've": 'there would have',
 "they'd've": 'they would have',
 "they'll've": 'they will have',
 "to've": 'to have',
 "we'd've": 'we would have',
 "we'll've": 'we will have',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "what're": 'what are',
 "what've": 'what have',
 "when's": 'when is',
 "when've": 'when have',
 "where'd": 'where did',
 "where've": 'where have',
 "who'll've": 'who will have',
 "why's": 'why is',
 "why've": 'why have',
 "will've": 'will have',
 "won't've": 'will not have',
 "wouldn't've": 'would not have',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "you'd've": 'you would have',
 "you'll've": 'you will have',
'bebecause':'be because',
'I’m':'I am',
              'it’s':'it is',
                 'I’ve':'I have',
                 'don’t':'do not',
                'However':'but',
                 'It’s':'It is',
                 'didn’t':'did not',
                 'can’t':'can not',
                 'that’s':'that is',
'doesn’t':'does not',
'I’d':'I had',
'isn’t':'is not',
'wasn’t':'was not'
                
                }

def wordreplace(tweet,replaceWords):
    for key in replaceWords:
        tweet = tweet.replace(key,replaceWords[key])
    return tweet

for index, row in train_df['all_text'].iteritems():
    train_df['all_text'][index] = wordreplace(row,replaceWords1)
    
for index, row in test_df['all_text'].iteritems():
    test_df['all_text'][index] = wordreplace(row,replaceWords1)

In [22]:
# Now lets check if we have improved on our coverage 
vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
oov[:20]


Found embeddings for 31.35% of vocab
Found embeddings for  81.41% of all text


[('[SEP]', 54711),
 ('),', 2306),
 ('STACKOVERFLOW', 1253),
 (');', 1009),
 ('=&gt;', 826),
 ('but,', 768),
 ('LIFE_ARTS', 709),
 ('&gt;', 676),
 ('example,', 573),
 ('this:', 537),
 ('(or', 505),
 ('-&gt;', 466),
 ('So,', 433),
 ('$$', 426),
 ('},', 420),
 ('&lt;', 415),
 ('(and', 390),
 ('this?', 383),
 ('is,', 371),
 ('(I', 339)]

To handle special characters and emoji’s, we will use an embedding
One of the thing to note is that your embeddings might contains few special characters embeddings which if you delete then will lost information.

So lets try and check how many embedding’s we have coverage

In [23]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list += "'"

In [24]:
glove_chars = ''.join([c for c in tqdm(glove_embeddings) if len(c) == 1])
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])
glove_symbols

',.":)(-!?|;$&/[]>%=#*+\\•~@£·_{}©^®`<→°€™›♥←×§″′█½…“★”–●►−¢²¬░¡¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■▀¨▄♫☆¯♦¤▲¸¾⋅∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤¹≤‡√◄━⇒▶º≥╝♡◊。✈≡☺✔↵≈✓♣☎℃◦└‟～！○◆№♠▌✿▸⁄□❖✦．÷｜┃／￥╠↩✭▐☼µ☻┐├«∼┌℉☮฿≦♬✧〉－⌂✖･◕※‖◀‰\x97↺∆┘┬╬،⌘⊂ª＞〈⎙Å？☠⇐▫∗∈≠♀ƒ♔˚℗┗＊┼❀＆∩♂‿∑‣➜┛⇓☯⊖☀┳；∇⇑✰◇♯☞´↔┏｡◘∂✌♭┣┴┓✨ˈ˜❥┫℠✒［∫\x93≧］\x94∀♛\x96∨◎ˑ↻⅓⇩＜≫✩ˆ✪♕؟₤☛╮␊＋┈ɡ％╋▽⇨┻⊗￡।▂✯▇＿➤₂✞＝▷△◙▅✝ﾟ∧␉☭┊╯☾➔∴\x92▃↳＾׳➢╭➡＠⊙☢˝⅛∏„①๑∥❝☐▆╱⋙๏☁⇔▔\x91②➚◡╰٠♢˙۞✘✮☑⋆ℓⓘ❒☣✉⌊➠∣❑⅔◢ⓒ\x80〒∕▮⦿✫✚⋯♩☂ˌ❞‗܂☜‾✜╲∘⟩＼⟨·⅜✗♚∅ⓔ◣͡‛❦⑨③◠✄❄１∃␣≪｢≅◯☽２∎｣⁰❧̅ǡⒶ↘⚓▣˘∪⇢✍⊥＃⅝⎯↠۩☰◥⊆✽ﬁ⚡↪ở❁☹◼☃◤❏ⓢ⊱α➝̣✡∠｀▴┤Ȃ∝♏ⓐ✎;３④␤＇❣⅞✂✤ⓞ☪✴⌒˛♒＄ɪ✶▻Ⓔ◌◈۲Ʈ❚ʿ❂￦◉╜̃ν✱╖❉₃ⓡℝ٤↗❶ʡ۰ˇⓣ♻➽۶₁ʃ׀✲ʤ✬☉▉≒☥⌐♨✕ⓝ⊰❘＂⇧̵➪４▁β۱▏⊃ⓛ‚♰́✏⏑̶٩Ⓢー⩾日￠❍≃⋰♋ɿ､̂❋✳ⓤ╤▕⌣✸℮⁺▨⑤╨Ⓥ♈❃☝５✻⊇≻♘♞◂７✟⌠✠☚✥❊ƂⒸ⌈❅Ⓡ♧Ⓞɑλ۵▭❱Ⓣ∟☕♺∵⍝ⓑɔ✵✣ℤ年ℕ٭♆Ⓘⅆ∶⚜◞்✹Ǥȡ➥ᴥ↕ɂ̳∷✋➧∋̿ͧʘ┅⥤⬆ǀμ₄⋱ʔ☄↖⋮۔♌Ⓛ╕♓ـ⁴❯♍▋✺⭐６✾♊➣▿Ⓑ♉Ａ⏠◾▹⑥⩽в↦╥⍵⌋։➨и∮⇥ⓗⒹ⁻ʊ⎝⌥⌉◔◑ǂ✼♎ℂ♐╪ɨ⊚☒⇤θВⓜ⎠Ｏ◐ǰ⚠╞ﬂ◗⎕ⓨ☟Ｉⓟ♟❈↬ⓓ◻♮❙а♤∉؛⁂例Ⓝ־♑╫╓╳⬅☔πɒɹ߂☸ɐʻ┄╧ʌ׃８ʒ⎢❆⋄⚫̏☏➞͂␙Ⓤ◟Ƥʕ̊Ȥ⚐✙は↙̾ωΔ℘ﾞ✷⑦φ⍺❌⊢▵✅ｗ９ⓖ☨▰ʹ╡Ⓜ☤∽╘˹↨ȿ♙⬇♱⌡Ω⠀╛❕┉Ⓟ̀Ǩ♖ⓚ┆⑧⎜ǹ◜⚾⤴✇╟⎛☩➲➟ⓥⒽ⏝◃０₀╢月↯✆˃⍴❇⚽╒Ｃɻɤ̸♜☓Ｔ➳⇄γ☬⚑✐⁵δȭ⌃◅▢ｓȸ❐∊☈ⅇℜ॥σ⎮ȣ▩のτεＳு⊹‵␔☊➸̌☿⇉➊⊳╙⁶ⓦ⇣｛̄↝⎟ℳ▍❗ℑＭɾｍ״Γ΄▞◁⛄⇝⎪ˤ♁ｖ⇠☇✊位ℒạி｝๐⭕➘Ｂ❺ɸˡ⁀⑩ｃ⅕Ƽ۳☙❛₆ƪ❓⟲Ʒ⇀≲Ｐ❷١ⓕ⎥Ｄс\u06ddǥͤ₋̱̎♝≳▙Ｒʹ➭ℰ܀ʺȫⒼ⇛ˉ▊❸号⇗̷

In [25]:
# Chars available in the embedding’s
jigsaw_chars = build_vocab(list(train_df["all_text"]))
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])
jigsaw_symbols

'?[]-(:,.)\n/!_"לאיקוםעדחבשכןטתרפנהגמסזף\xa0>=#;&$\\%{}@`|+^~ДрузьяиОкжентошКаБсФыхвТчЛйРЯЖЗдСпᴀᴜʟᴛᴇʀ“¹–²\u202f*…≈—”\u3000⁄×°ﬂ£शेखरٍ：ε∀∧π\uf3c7Ƭʜι√φθ←→↑ᴚɯɐˆ\ue806\ue805\ue804\ue800\ue801\ue802\ue803−ИПМマーチンɔʌɑامریه捨てる神あれば、拾うり英字ことえ環境設定を表示ц€ˈɪʃ安宇（）•µºךﬀ«»©<ᵀᴹձոգչ\t⁺⁻Ծիրւնյաªツ♭♯大易归真´⌥？筲箕灣東街號地下九龍觀塘偉業振邦工廈鋪土瓜旭日一瑞第期座ΦְִֵֶַָֹֻּׁѪʊɹʒˌ★웃준영αғη™®βֱֲצ\u200bϵϕλΔ∈∞ψ⌘¬∨حفظ≤♦―‑ɒːɝ\u200eΨ÷ʔ⅓⅔ֽׂ֧֣־₂μ►‒±\xad└─┌˚‐ץΕύοΧκίδﬁɃḁのエデ挿入歌「」ꘌ·\ufeff͢ωρνʍɥʇ‹›ℝχγυ⁰ɴ′∗◇⁵₄⁹₁花咲かせ－'

In [26]:
# Basically we can delete all symbols we have no embeddings for:
symbols_to_delete = ''.join([c for c in jigsaw_symbols if not c in glove_symbols])
symbols_to_delete

'\nלאיקוםעדחבשכןטתרפנהגמסזף\xa0ДрузьяОкжентошКБФыхТчЛйРЯЖЗдСпᴀᴜʟᴛᴇʀ\u202f\u3000शेखरٍ\uf3c7Ƭʜιᴚ\ue806\ue805\ue804\ue800\ue801\ue802\ue803ИПМマチンامریه捨てる神あれば拾うり英字ことえ環境設定を表示ц安宇ךﬀᵀᴹձոգչ\tԾիրւնյաツ大易归真筲箕灣東街號地下九龍觀塘偉業振邦工廈鋪土瓜旭一瑞第期座ְִֵֶַָֹֻּׁѪ웃준영ғֱֲצ\u200bϵϕψحفظ‑ɝ\u200eΨֽׂ֧֣‒\xad‐ץΕύοΧκίɃḁエデ挿入歌「」ꘌ\ufeff͢ʍɥʇχυɴ⁹花咲かせ'

In [27]:
# The symbols we want to keep we need to isolate from our words. So lets setup a list of those to isolate.
symbols_to_isolate = ''.join([c for c in jigsaw_symbols if c in glove_symbols])
symbols_to_isolate

'?[]-(:,.)/!_">=#;&$\\%{}@`|+^~иасв“¹–²*…≈—”⁄×°ﬂ£：ε∀∧π√φθ←→↑ɯɐˆ−ーɔʌɑ、€ˈɪʃ（）•µº«»©<⁺⁻ª♭♯´⌥？日Φʊɹʒˌ★αη™®βλΔ∈∞⌘¬∨≤♦―ɒː÷ʔ⅓⅔־₂μ►±└─┌˚δﬁの·ωρν‹›ℝγ⁰′∗◇⁵₄₁－'

In [28]:
# Note : Next comes the next trick. Instead of using an inefficient loop of replace we use translate. 


isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

In [29]:
#So lets apply that function to our text and reasses the coverage

train_df['all_text'] = train_df['all_text'].progress_apply(lambda x:handle_punctuation(x))
test_df['all_text'] = test_df['all_text'].progress_apply(lambda x:handle_punctuation(x))

In [30]:
vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
oov[:20]


Found embeddings for 68.63% of vocab
Found embeddings for  98.00% of all text


[('STACKOVERFLOW', 1253),
 ('Bebecause', 156),
 ("Let's", 139),
 ('askubuntu', 128),
 ("'field", 97),
 ('myStaticIntStack', 76),
 ('CURLOPT', 72),
 ('instrumentsettingsid', 71),
 ('stackSize', 69),
 ('storedElements', 65),
 ("user's", 65),
 ('RippleShaderProgram', 64),
 ('aStack', 64),
 ('Appium', 63),
 ("'class'", 60),
 ("'Roboto", 56),
 ('jsonobject', 56),
 ('brmfc', 56),
 ("name'", 51),
 ("'px'", 50)]

In [31]:
# Lets apply pre-processing before we tokenize the words

for index, row in train_df['all_text'].iteritems():
    train_df['all_text'][index] = pre_process(row)
for index, row in test_df['all_text'].iteritems():
    test_df['all_text'][index] = pre_process(row)

In [32]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

In [33]:
def handle_contractions(x):
    x = tokenizer.tokenize(x)
    x = ' '.join(x)
    return x


In [34]:
train_df['all_text'] = train_df['all_text'].progress_apply(lambda x:handle_contractions(x))
test_df['all_text'] = test_df['all_text'].progress_apply(lambda x:handle_contractions(x))

In [35]:
# Lets check after we have tokenize

vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embeddings)
oov[:20]


Found embeddings for 73.08% of vocab
Found embeddings for  98.04% of all text


[('STACKOVERFLOW', 1253),
 ('Bebecause', 156),
 ('askubuntu', 128),
 ('myStaticIntStack', 76),
 ('CURLOPT', 72),
 ('instrumentsettingsid', 71),
 ('stackSize', 69),
 ('storedElements', 65),
 ('RippleShaderProgram', 64),
 ('aStack', 64),
 ('Appium', 63),
 ('jsonobject', 56),
 ('brmfc', 56),
 ('setab', 48),
 ('setaf', 48),
 ('PlainSocketImpl', 48),
 ('luvut', 48),
 ('Bodycopy', 47),
 ('producttypeid', 46),
 ('ShaderProgramConstants', 46)]

In [36]:
def fix_quote(x):
    x = [x_[1:] if x_.startswith("'") else x_ for x_ in x]
    x = ' '.join(x)
    return x

In [37]:
train_df['all_text'] = train_df['all_text'].progress_apply(lambda x:fix_quote(x.split()))
test_df['all_text'] = test_df['all_text'].progress_apply(lambda x:fix_quote(x.split()))

In [38]:
vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embeddings)
oov[:10]


Found embeddings for 73.08% of vocab
Found embeddings for  98.04% of all text


[('STACKOVERFLOW', 1253),
 ('Bebecause', 156),
 ('askubuntu', 128),
 ('myStaticIntStack', 76),
 ('CURLOPT', 72),
 ('instrumentsettingsid', 71),
 ('stackSize', 69),
 ('storedElements', 65),
 ('RippleShaderProgram', 64),
 ('aStack', 64)]

In [39]:
# Lets also check test data has equal coverage
vocab = build_vocab(list(test_df['all_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embeddings)
oov[:10]


Found embeddings for 86.50% of vocab
Found embeddings for  97.93% of all text


[('STACKOVERFLOW', 103),
 ('CURLOPT', 26),
 ('bmAttributes', 21),
 ('testimonialList', 20),
 ('bInterval', 14),
 ('wMaxPacketSize', 14),
 ('bEndpointAddress', 14),
 ('testimonialService', 13),
 ('cyvo', 13),
 ('datLoad', 12)]

Checking CRAWL embeddings Coverage

In [40]:
tic = time.time()
crawl_embeddings = load_embeddings(CRAWL_EMBEDDING_PATH)
print(f'loaded {len(glove_embeddings)} word vectors in {time.time()-tic}s')

loaded 2196008 word vectors in 11.940247774124146s


In [41]:
vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,crawl_embeddings)
oov[:20]


Found embeddings for 72.89% of vocab
Found embeddings for  95.19% of all text


[('``', 35179),
 ('_', 20974),
 ('STACKOVERFLOW', 1253),
 ('`', 397),
 ('usepackage', 294),
 ('serverfault', 223),
 ('Bebecause', 156),
 ('askubuntu', 128),
 ('dfrac', 116),
 ('tikzpicture', 92),
 ('myStaticIntStack', 76),
 ('graphicdesign', 76),
 ('setopt', 72),
 ('instrumentsettingsid', 71),
 ('stackSize', 69),
 ('storedElements', 65),
 ('RippleShaderProgram', 64),
 ('aStack', 64),
 ('newcommand', 63),
 ('jsonobject', 56)]

In [42]:
punctuation = '_`'

train_df['all_text'] = train_df['all_text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))
test_df['all_text'] = test_df['all_text'].apply(lambda x: ''.join(ch for ch in x if ch not in set(punctuation)))

In [43]:
#Lets check the embeddings now

vocab = build_vocab(list(train_df['all_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,crawl_embeddings)
oov[:10]


Found embeddings for 72.90% of vocab
Found embeddings for  97.87% of all text


[('STACKOVERFLOW', 1253),
 ('usepackage', 294),
 ('serverfault', 223),
 ('Bebecause', 156),
 ('askubuntu', 128),
 ('dfrac', 116),
 ('tikzpicture', 92),
 ('myStaticIntStack', 76),
 ('graphicdesign', 76),
 ('setopt', 72)]

In [44]:
X = train_df['all_text']
y_columns = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written', 'answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']
#y = train_df['question_asker_intent_understanding']
y = train_df[y_columns]
#y = train_df['question_fact_seeking']
test_pred = test_df['all_text']


In [45]:
train_df[y_columns].head(n=10)

,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,question_not_really_a_question,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,1.000000,0.333333,0.000000,0.000000,0.0,0.0,1.000000,1.000000,0.000000,0.0,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,0.000000,1.000000
1,1.000000,1.000000,0.000000,0.500000,1.0,1.0,0.444444,0.444444,0.666667,0.0,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.000000,0.000000,0.666667,0.888889
2,0.888889,0.666667,0.000000,1.000000,1.0,1.0,0.666667,0.444444,0.333333,0.0,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.000000,0.333333,1.000000,0.888889
3,0.888889,0.666667,0.666667,1.000000,1.0,1.0,0.444444,0.444444,0.000000,0.0,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.000000,0.000000,1.000000,1.000000
4,1.000000,0.666667,0.000000,1.000000,1.0,1.0,0.666667,0.666667,0.000000,0.0,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.000000,0.000000,1.000000,1.000000
5,1.000000,0.666667,0.000000,0.500000,1.0,1.0,0.444444,0.333333,0.000000,0.0,...,0.833333,0.888889,0.666667,0.888889,1.000000,0.733333,0.666667,0.666667,0.000000,0.777778
6,0.888889,0.333333,0.000000,0.000000,1.0,1.0,0.444444,0.333333,1.000000,0.0,...,0.666667,1.000000,0.777778,1.000000,0.888889,0.866667,1.000000,0.000000,1.000000,0.888889
7,1.000000,1.000000,0.000000,0.000000,1.0,1.0,0.888889,0.777778,0.333333,0.0,...,1.000000,0.666667,0.666667,0.888889,1.000000,0.866667,0.666667,0.000000,0.000000,0.888889
8,1.000000,0.333333,0.000000,1.000000,1.0,1.0,0.666667,0.555556,0.000000,0.0,...,0.888889,0.666667,0.333333,0.666667,0.666667,0.266667,0.000000,0.000000,0.000000,0.888889
9,1.000000,0.777778,0.000000,0.333333,1.0,1.0,0.777778,0.888889,0.666667,0.0,...,1.000000,0.777778,0.333333,0.777778,0.777778,0.533333,0.333333,0.000000,0.666667,0.777778


In [46]:
MAX_LEN = 100
max_features = 500000

In [47]:

# Its really important that you intitialize the keras tokenizer correctly. Per default it does lower case and removes a lot of symbols. We want neither of that!

tokenizer = text.Tokenizer(num_words = max_features, filters='',lower=False)

In [48]:
tokenizer.fit_on_texts(list(X) + list(test_pred))

In [49]:
import numpy as np
crawl_matrix, unknown_words_crawl = build_matrix(tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

glove_matrix, unknown_words_glove = build_matrix(tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

max_features = max_features or len(tokenizer.word_index) + 1
max_features

embedding_matrix = np.concatenate([crawl_matrix, glove_matrix], axis=-1)
embedding_matrix.shape

import gc
del crawl_matrix
del glove_matrix
gc.collect()

n unknown words (crawl):  19465
n unknown words (glove):  19354


0

In [50]:
X = tokenizer.texts_to_sequences(X)
test_pred = tokenizer.texts_to_sequences(test_pred)

In [51]:
X = sequence.pad_sequences(X, maxlen=MAX_LEN)
test_pred = sequence.pad_sequences(test_pred, maxlen=MAX_LEN)

In [52]:
checkpoint_predictions = []
weights = []

In [53]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate,Flatten,Lambda
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D,PReLU,LSTM
from keras.preprocessing import text, sequence
from keras.callbacks import LearningRateScheduler
from keras.models import Sequential
from keras.preprocessing import text, sequence
from keras import regularizers
import keras
import tensorflow as tf
import keras.backend as K
from sklearn.model_selection import train_test_split
from keras.engine.topology import Layer
import tensorflow_hub as hub
from keras.layers.normalization import BatchNormalization

In [54]:
#### testing delete

In [55]:
X_train , X_val, y_train  , y_val = train_test_split(X , 
                                                     y.values , 
                                                     #stratify = y.values, 
                                                     train_size = 0.8,
                                                     random_state = 100)

In [56]:
y_train.shape

(4863, 30)

In [57]:
embedding_matrix.shape

(69319, 600)

In [58]:
len(X_train[1])

100

In [59]:
from scipy.stats import spearmanr, rankdata
from keras.callbacks import Callback
# Compatible with tensorflow backend
class SpearmanRhoCallback(Callback):
    def __init__(self, training_data, validation_data, patience, model_name):
        self.x = training_data[0]
        self.y = training_data[1]
        self.X_val = validation_data[0]
        self.y_val = validation_data[1]
        
        self.patience = patience
        self.value = -1
        self.bad_epochs = 0
        self.model_name = model_name

    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.X_val)
        ind = 0
        rho_val = np.mean([spearmanr(self.y_val[:, ind], y_pred_val[:, ind] + np.random.normal(0, 1e-7, y_pred_val.shape[0])).correlation for ind in range(y_pred_val.shape[1])])
        if rho_val >= self.value:
            self.value = rho_val
        else:
            self.bad_epochs += 1
        if self.bad_epochs >= self.patience:
            print("Epoch %05d: early stopping Threshold" % epoch)
            self.model.stop_training = True
            #self.model.save_weights(self.model_name)
        print('\rval_spearman-rho: %s' % (str(round(rho_val, 4))), end=100*' '+'\n')
        return rho_val

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [60]:
maxlen = 100
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
num_class = len(np.unique(y))
def build_model(embedding_matrix, num_aux_targets):
        model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Conv1D(150, 5, activation='relu'),
        tf.keras.layers.LSTM(128, return_sequences=True),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(units=120,activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(30, activation='sigmoid')])
        model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
             loss = 'binary_crossentropy',
             metrics = ['accuracy'])
        print(model.summary())
        return model                         

In [61]:
from keras.callbacks import EarlyStopping 
es = EarlyStopping(monitor='val_loss', mode ='min' ,verbose =1,patience=0.1)
NUM_MODELS =2
for model_idx in range(NUM_MODELS):
    model = build_model(embedding_matrix,1)
    for global_epoch in range(200):
         model.fit(X_train,
                   y_train,
                   epochs=200,
                   verbose=1,
                   batch_size = 50,
                   validation_data=(X_val, y_val),
                   class_weight ="temporal",#class_weights,
                   callbacks=[SpearmanRhoCallback(training_data=(X_train, y_train), validation_data=(X_val, y_val),
                                       patience=5, model_name=f'pre_trained.hdf5'),
                              LearningRateScheduler(lambda epoch: 1e-3 * (0.4 ** global_epoch)),
                       es
                   ]
                  )
                   
    checkpoint_predictions.append(model.predict(test_pred))
    weights.append(3 ** global_epoch)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 600)         41591400  
_________________________________________________________________
dropout (Dropout)            (None, None, 600)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 150)         450150    
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         142848    
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               15480     
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0

In [62]:
score = model.evaluate(X_val, y_val, verbose=1)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

1216/1216 [==============================] - 3s 2ms/sample - loss: 0.4052 - accuracy: 0.5668
Test Score: 0.4052015244960785
Test Accuracy: 0.56677634


In [63]:
predictions = np.average(checkpoint_predictions,  axis=0) 

In [64]:
predictions

array([[0.899848  , 0.66241944, 0.04432467, ..., 0.07429039, 0.83859503,
        0.89074814],
       [0.88317   , 0.51526725, 0.00369965, ..., 0.1339152 , 0.22252591,
        0.91686624],
       [0.8732196 , 0.5784099 , 0.06105335, ..., 0.09912629, 0.9032054 ,
        0.90208197],
       ...,
       [0.92035294, 0.68149215, 0.05191036, ..., 0.1019247 , 0.1868532 ,
        0.89883   ],
       [0.9349798 , 0.6864717 , 0.1297013 , ..., 0.05913281, 0.7938149 ,
        0.93935376],
       [0.82836306, 0.5301217 , 0.02356471, ..., 0.19862287, 0.4550416 ,
        0.87162805]], dtype=float32)

In [65]:
sample_submission_df.iloc[:, 1:] = predictions

In [66]:
sample_submission_df.head(n=6)

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.899848,0.662419,0.044325,0.722090,0.898142,0.874400,0.628994,0.547596,0.349768,...,0.836232,0.887013,0.625625,0.952550,0.958107,0.824240,0.047764,0.074290,0.838595,0.890748
1,46,0.883170,0.515267,0.003700,0.726470,0.724975,0.901990,0.543434,0.428468,0.149733,...,0.734560,0.948880,0.641839,0.972765,0.984354,0.879297,0.928506,0.133915,0.222526,0.916866
2,70,0.873220,0.578410,0.061053,0.530529,0.871389,0.792842,0.593119,0.512243,0.418779,...,0.806161,0.913471,0.628519,0.956350,0.961888,0.826709,0.071427,0.099126,0.903205,0.902082
3,132,0.885719,0.554728,0.004211,0.750892,0.821368,0.938581,0.552036,0.428860,0.237048,...,0.774463,0.949267,0.688252,0.968126,0.982747,0.888655,0.804738,0.166936,0.504031,0.917942
4,200,0.872063,0.610370,0.021083,0.681417,0.870358,0.876443,0.587200,0.534437,0.323158,...,0.796707,0.887699,0.648207,0.948172,0.945943,0.822085,0.358268,0.145560,0.590179,0.888142
5,245,0.932680,0.665206,0.059654,0.750859,0.822914,0.780561,0.648823,0.597958,0.245079,...,0.863070,0.925651,0.647871,0.965833,0.970282,0.836527,0.080609,0.049701,0.752412,0.925128


In [67]:
sample_submission_df.to_csv("submission.csv", index= False)